In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, BatchNormalization, Input
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
#from tensorflow.keras.utils import np_utils
import os
#import cv2
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.imagenet_utils import preprocess_input, decode_predictions
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing.image import load_img
import matplotlib.pyplot as plt

In [2]:
width_shape = 224
height_shape = 224
num_classes = 3
epochs = 50
batch_size = 32

In [3]:
#image_input = Input(shape=(width_shape, height_shape, 3))

#modelVGG16raw = VGG16(input_tensor=image_input, include_top=True,weights='imagenet')

#modelVGG16raw.summary()

In [4]:
image_input = Input(shape=(width_shape, height_shape, 3))
custom_model = VGG16(input_tensor=image_input, include_top=True,weights='imagenet')

last_layer = custom_model.get_layer('block5_pool').output
x= Flatten(name='flatten')(last_layer)
x = Dense(1024, activation='relu', name='fc1')(x)
x = Dense(1024, activation='relu', name='fc2')(x)
out = Dense(num_classes, activation='softmax', name='outputECG')(x)
custom_model = Model(image_input, out)
#custom_model.summary()

# freeze all the layers except the dense layers
for layer in custom_model.layers[:-3]:
	layer.trainable = False

custom_model.summary()

#metricF1 = tfa.metrics.F1Score(num_classes=3, threshold=0.5)

custom_model.compile(loss="sparse_categorical_crossentropy",
                      optimizer="sgd", metrics=["accuracy"])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [5]:
IMG_WIDTH=224
IMG_HEIGHT=224

In [6]:
def convert_numpy_array(y_class_name):
    target_dict={k: v for v, k in enumerate(np.unique(y_class_name))}
    target_val = [target_dict[y_class_name[i]] for i in range(len(y_class_name))]
    y = np.array(target_val)
    return y

In [7]:
img_data_array =[]
class_name = []

def create_dataset(img_folder): 
    for dir1 in os.listdir(img_folder):
        for file in os.listdir(os.path.join(img_folder, dir1)):
            image_path= os.path.join(img_folder, dir1,  file)
            splitted = str(image_path.lower()).rsplit('.')
            extension = splitted[-1]
            if extension == "png": 
                image = load_img(image_path,color_mode='rgb',target_size=None,interpolation='nearest')
                image = np.array(image)
                image = image.astype('float32')
                image /= 255 
                #image = np.expand_dims(image, axis = 0)
                img_data_array.append(image)
                class_name.append(dir1)

    return img_data_array, class_name

In [8]:
#!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1U9snFKJ3yc3liETDK5BHFt0BvrZQ8Cio' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1U9snFKJ3yc3liETDK5BHFt0BvrZQ8Cio" -O IMGV5.zip && rm -rf /tmp/cookies.txt

In [9]:
#import shutil
#!mkdir IMG5
#!unzip 'IMGV5.zip' -d IMG5
#shutil.rmtree('IMG5')

In [10]:
x_train, y_train = create_dataset('IMG5')

len(y_train)

18000

In [11]:
len(y_train)
y_train = convert_numpy_array(y_train)


In [26]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [13]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.20, random_state=64)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.20, random_state=64)

In [14]:
version = 'v1.0'
log_dir='logs/cnnVGG16_az_'+ version
tensorboardCNN = TensorBoard(log_dir,histogram_freq = 1)

In [15]:
custom_model.fit(x = np.array(x_train, np.float32)
                ,y = np.array(list(map(int,y_train)), np.float32)
                ,batch_size= 32
                ,validation_data=(np.array(x_val, np.float32),  np.array(list(map(int,y_val)), np.float32))
                ,epochs= 100
                ,verbose = 1
                ,callbacks=[tensorboardCNN])

Epoch 1/100
360/360 [==============================] - 153s 415ms/step - loss: 0.8216 - accuracy: 0.6093 - val_loss: 0.6169 - val_accuracy: 0.7229
Epoch 2/100
360/360 [==============================] - 194s 538ms/step - loss: 0.6152 - accuracy: 0.7260 - val_loss: 0.6372 - val_accuracy: 0.7097
Epoch 3/100
360/360 [==============================] - 193s 538ms/step - loss: 0.5498 - accuracy: 0.7522 - val_loss: 0.5567 - val_accuracy: 0.7375
Epoch 4/100
360/360 [==============================] - 193s 537ms/step - loss: 0.5083 - accuracy: 0.7805 - val_loss: 0.4782 - val_accuracy: 0.7861
Epoch 5/100
360/360 [==============================] - 193s 537ms/step - loss: 0.4826 - accuracy: 0.7856 - val_loss: 0.4148 - val_accuracy: 0.8333
Epoch 6/100
360/360 [==============================] - 194s 538ms/step - 

In [16]:
#score = custom_model.evaluate(np.array(x_test, np.float32), np.array(list(map(int,y_test)), np.float32), verbose = 1) 

#print('array:', len(score))
#print('Test loss:', score[0]) 
#print('Test accuracy:', score[1])

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [17]:
tf.saved_model.save(custom_model, '../modelnew/')

INFO:tensorflow:Assets written to: ../modelnew/assets


In [20]:
tf.saved_model.load(custom_model, '../modelnew/save_model.pb')

TypeError: Expected binary or unicode string, got <keras.engine.functional.Functional object at 0x7f24587f1b50>

In [21]:
tf.keras.experimental.export_saved_model(custom_model, '../modelv2/')

AttributeError: module 'keras.api._v2.keras.experimental' has no attribute 'export_saved_model'

In [24]:
tf.saved_model.save(custom_model, './outputs/modelv2/')

INFO:tensorflow:Assets written to: ./outputs/modelv2/assets


In [23]:
tf.saved_model.load('./model')

OSError: SavedModel file does not exist at: ./model/{saved_model.pbtxt|saved_model.pb}